Business Intelligence

Time Series Analysis
- Are there monthly and weekly sales seasonality?
- Are there anomalous* peaks and troughs - what could explain them?
- How does every year from 2015-2019 compare against each other?
  
Distribution and Aggregations
- What's the distribution of sales per product category^? 
- What's the dollar value per month/year per:
  - individual
  - cluster (once identified)
  - product category

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# member = pd.read_csv("cl_member.csv")
# guest = pd.read_csv("cl_guest.csv")

In [4]:
df = member[["acn","timestamp","item","item_group","total","sys_id"]]
df = df.rename(columns={"acn":"id","sys_id":"product_category"})

In [5]:
df2 = guest[["sys_id","timestamp","item","item_group","total"]]
df2 = df2.rename(columns={"sys_id":"id"})
df2["product_category"] = "Guest"

In [6]:
len(df)+len(df2)

143718

**Task: Concat df and df2**  
But first, check if ids are unique, if not find out why not.

In [7]:
member.head()

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,7315,2015-01-03 11:00:00,Pia Lomax,Lessons with Joe Snailum (01/03/2015),Lessons,Paid,1.0,65.00,0.00,0.00,65.00,65.00,0.0,0.0,79,General Single Membership
1,1,37831,2015-08-12 21:30:00,Jennifer Hernandez,$1 Snacks,Snacks,Paid,1.0,0.92,0.00,0.08,1.00,1.00,0.0,0.0,566,General Family Membership
2,2,37844,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
3,3,37843,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
4,4,37842,2015-08-13 10:50:00,Douglas Draeger,Mango Tango,Smoothie,Paid,1.0,4.39,0.44,0.33,4.28,4.28,0.0,0.0,1532,General Property Owner


In [8]:
df[df.id.isin(df2.id)].id.count()

971

In [9]:
dups = df[df.id.isin(df2.id)].id.unique()

In [10]:
member[member.acn.isin(dups)].name.unique()

array(['Ashley Lloyd', 'Lee Stacy', 'Rhonda Stacy', 'Diego Garcia',
       'Hartley Clark', 'Evyn Clark', 'Abigail Kelly', 'Cole Jecmenek',
       'David Smith', 'Khloe Villa', 'Aaron Malakoff', 'Jessica Villa',
       'Ryan Villa'], dtype=object)

Check from the cancellation data.

In [11]:
# cancel = pd.read_csv("cancellation.csv")
cancel[cancel["Last name"]=="Malakoff"]

NameError: name 'cancel' is not defined

Duplicated entries who are **confirmed** in the Cancellation data

- Aaron Malakoff  
- Rhonda, Lee Stacy  
- David Smith

Duplicated entries who have **similar names** in the Cancellation data
- Roberto same as Diego Garcia?  
- Wes same as Hartley/Evyn Clark?  
- Jarrett same as Cole Jecmenek?

Duplicated entries who are **not** in the Cancellation data
- Ashley Lloyd  
- Abigail Kelly
- Jessica Villa

In [12]:
member.item_group.unique().tolist()

['Lessons',
 'Snacks',
 'Smoothie',
 'White Wine',
 'Drinks',
 'Tips',
 'Guest Fee',
 'Apparel',
 'Accessories',
 'Tennis Balls',
 'Adult Clinics',
 'Massage',
 'Membership Initiation ',
 'Membership Dues',
 'Junior Clinics',
 'Beer',
 'Date Night',
 'Racquet Rental',
 'Red Wine',
 'String',
 'Package Liability',
 'Restring/ Labor Only',
 'Special Event',
 'Shoes',
 'Facility Rental',
 'Dinners',
 'Camp',
 'Racquets',
 'Leagues',
 'Personal Training',
 'Catie Johnson Home School',
 'Fitness Special Event',
 'Pro Shop',
 'Gift Certificate Liability',
 'Social Event',
 'Swim Lessons ',
 'Credits',
 'Swim Camp',
 'TRAIN4LIFE',
 'Mini Bees Soccer',
 'Teen Performance Camp',
 'CDM- MISC',
 'Soccer- MSOS',
 'Special Event-Non Taxable (Includes Instruction)',
 'Food Allowance',
 'Naturally Cafe',
 'Naturally-Salad',
 'Naturally Lunch Special',
 'Naturally Juices',
 'Naturally-Smoothies',
 'Naturally- Sandwiches',
 'Court Time',
 'VIP TENNIS',
 'VIP FITNESS',
 'Food & Beverage',
 'Kid Fit',
 '

In [20]:
member[member.item_group=="Membership Initiation "]

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
50,50,37784,2015-08-12 17:37:00,Kristi Hayduk,Membership Initiation,Membership Initiation,Paid,1.0,900.00,0.00,74.25,974.25,974.25,0.0,0.0,1628,General Single Membership
224,224,37566,2015-08-09 14:05:00,Brett Kelley,Membership Initiation,Membership Initiation,Paid,1.0,1200.00,0.00,99.00,1299.00,1299.00,0.0,0.0,1614,General Family Membership
662,662,38335,2015-08-20 11:21:00,Scott Wilk,Wilk property transfer,Membership Initiation,Paid,1.0,1200.00,99.00,99.00,1200.00,1200.00,0.0,0.0,1623,General Property Owner
1298,1298,37110,2015-08-04 14:45:00,Johnathan Burda,Membership Initiation,Membership Initiation,Paid,1.0,1200.00,0.00,99.00,1299.00,1299.00,0.0,0.0,1607,General Family Membership
1628,1628,40663,2015-09-01 09:35:00,Kenneth Dennis,Membership Initiation,Membership Initiation,Paid,1.0,900.00,0.00,74.25,974.25,974.25,0.0,0.0,1630,General Single Membership
2396,2396,38986,2015-08-25 11:05:00,Debbie Wedlake,general single,Membership Initiation,Paid,1.0,900.00,0.00,74.25,974.25,974.25,0.0,0.0,1644,General Single Membership
2434,2434,38919,2015-08-24 19:12:00,Paige Leinneweber,Leinneweber intiation fee,Membership Initiation,Paid,1.0,1200.00,0.00,99.00,1299.00,1299.00,0.0,0.0,1641,General Family Membership
3917,3917,32000,2015-07-01 10:03:00,Michael Singleton,General family Initiation,Membership Initiation,Paid,1.0,1200.00,0.00,99.00,1299.00,1299.00,0.0,0.0,1523,General Family Membership
3981,3981,31980,2015-07-01 08:38:00,Karen Truesdell-Bierman,single membership,Membership Initiation,Paid,1.0,900.00,0.00,74.25,974.25,974.25,0.0,0.0,1522,General Family Membership
3983,3983,31980,2015-07-01 08:38:00,Karen Truesdell-Bierman,single membership,Membership Initiation,Paid,1.0,900.00,0.00,74.25,974.25,974.25,0.0,0.0,1522,General Single Membership


In [30]:
member[member.name=="Cara Berry"].sort_values(by=["timestamp", "total"])

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
101782,2173,223509,2019-08-12 15:35:00,Cara Berry,Tango 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,3170,General Family Membership
99957,348,224447,2019-08-19 09:38:00,Cara Berry,Membership for &quot;General Family Membership...,Membership Dues,Paid,1.0,57.87,0.0,4.77,62.64,62.64,0.0,0.00,3170,General Family Membership
99959,350,224448,2019-08-19 09:38:00,Cara Berry,6 for 360 Tennis (6 pack),Package Liability,Paid,1.0,360.00,0.0,0.00,360.00,360.00,0.0,0.00,3170,General Family Membership
99738,129,224744,2019-08-21 13:56:00,Cara Berry,$3.00 Ice Cream,Snacks,Paid,1.0,2.77,0.0,0.23,3.00,3.00,0.0,0.00,3170,General Family Membership
99745,136,224745,2019-08-21 13:56:00,Cara Berry,Tango 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,3170,General Family Membership
99748,139,224747,2019-08-21 14:08:00,Cara Berry,$3.00 Ice Cream,Snacks,Paid,1.0,2.77,0.0,0.23,3.00,3.00,0.0,0.00,3170,General Family Membership
100486,877,225804,2019-08-29 10:54:00,Cara Berry,Adult Guest Fee,Guest Fee,Paid,1.0,10.00,0.0,0.83,10.83,10.83,0.0,0.00,3170,General Family Membership
103544,3935,225955,2019-08-29 18:55:00,Cara Berry,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.00,0.0,0.00,360.00,360.00,0.0,0.00,3170,General Family Membership
104302,4693,226798,2019-08-31 00:00:00,Cara Berry,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.00,0.0,0.00,20.00,20.00,0.0,0.00,3170,General Family Membership
103370,3761,226490,2019-08-31 00:00:00,Cara Berry,Membership for &quot;General Family Membership...,Membership Dues,Paid,1.0,138.00,0.0,11.39,149.39,149.39,0.0,0.00,3170,General Family Membership


In [35]:
member[member.item_group=="Lessons"]

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,7315,2015-01-03 11:00:00,Pia Lomax,Lessons with Joe Snailum (01/03/2015),Lessons,Paid,1.0,65.00,0.00,0.0,65.00,65.00,0.00,0.0,79,General Single Membership
52,52,37780,2015-08-13 19:30:00,Colton Wilk,Lessons with Jerel Pleasant (08/13/2015),Lessons,Paid,1.0,65.00,0.00,0.0,65.00,65.00,0.00,0.0,1626,General Property Owner
55,55,37774,2015-08-12 15:52:00,Mike Shellman,Lessons with Joe Snailum (08/08/2015),Lessons,Paid,1.0,55.00,0.00,0.0,55.00,55.00,0.00,0.0,127,General Family Membership
61,61,37763,2015-08-12 14:00:00,Matt Palmer,Lessons with Jerel Pleasant (08/12/2015),Lessons,Paid,1.0,65.00,0.00,0.0,65.00,65.00,0.00,0.0,197,General Property Owner
62,62,37763,2015-08-12 14:00:00,Matt Palmer,Lessons with Jerel Pleasant (08/12/2015),Lessons,Paid,1.0,65.00,0.00,0.0,65.00,65.00,0.00,0.0,197,General Family Membership
101,101,37980,2015-08-16 19:46:00,Lara Perry,Lessons with Luke Barker (08/16/2015),Lessons,Paid,1.0,49.50,0.00,0.0,49.50,49.50,0.00,0.0,1173,Premier Family Membership
111,111,37984,2015-08-18 08:30:00,Samantha Kadish,Lessons with Cal Loveridge (08/18/2015),Lessons,Paid,1.0,65.00,0.00,0.0,65.00,65.00,0.00,0.0,1054,General Property Owner
114,114,38031,2015-08-19 09:00:00,Lisa Hooker,Lessons with Joe Snailum (08/19/2015),Lessons,Paid,1.0,22.50,0.00,0.0,22.50,22.50,0.00,0.0,937,General Family Membership
115,115,38027,2015-08-18 08:30:00,Mike Shellman,Lessons with Joe Snailum (08/18/2015),Lessons,Paid,1.0,55.00,0.00,0.0,55.00,55.00,0.00,0.0,127,General Family Membership
116,116,38025,2015-08-18 07:30:00,Catherine Shellman,Lessons with Joe Snailum (08/18/2015),Lessons,Paid,1.0,55.00,0.00,0.0,55.00,55.00,0.00,0.0,636,General Family Membership


In [39]:
member[member.name=="Cara Berry"].sort_values(by=["timestamp", "total"])

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
101782,2173,223509,2019-08-12 15:35:00,Cara Berry,Tango 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,3170,General Family Membership
99957,348,224447,2019-08-19 09:38:00,Cara Berry,Membership for &quot;General Family Membership...,Membership Dues,Paid,1.0,57.87,0.0,4.77,62.64,62.64,0.0,0.00,3170,General Family Membership
99959,350,224448,2019-08-19 09:38:00,Cara Berry,6 for 360 Tennis (6 pack),Package Liability,Paid,1.0,360.00,0.0,0.00,360.00,360.00,0.0,0.00,3170,General Family Membership
99738,129,224744,2019-08-21 13:56:00,Cara Berry,$3.00 Ice Cream,Snacks,Paid,1.0,2.77,0.0,0.23,3.00,3.00,0.0,0.00,3170,General Family Membership
99745,136,224745,2019-08-21 13:56:00,Cara Berry,Tango 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,3170,General Family Membership
99748,139,224747,2019-08-21 14:08:00,Cara Berry,$3.00 Ice Cream,Snacks,Paid,1.0,2.77,0.0,0.23,3.00,3.00,0.0,0.00,3170,General Family Membership
100486,877,225804,2019-08-29 10:54:00,Cara Berry,Adult Guest Fee,Guest Fee,Paid,1.0,10.00,0.0,0.83,10.83,10.83,0.0,0.00,3170,General Family Membership
103544,3935,225955,2019-08-29 18:55:00,Cara Berry,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.00,0.0,0.00,360.00,360.00,0.0,0.00,3170,General Family Membership
104302,4693,226798,2019-08-31 00:00:00,Cara Berry,Monthly Food and Beverage Allowance September ...,Food Allowance,Paid,1.0,20.00,0.0,0.00,20.00,20.00,0.0,0.00,3170,General Family Membership
103370,3761,226490,2019-08-31 00:00:00,Cara Berry,Membership for &quot;General Family Membership...,Membership Dues,Paid,1.0,138.00,0.0,11.39,149.39,149.39,0.0,0.00,3170,General Family Membership


In [65]:
member[member.item_group=="Naturally-Smoothies"].sort_values(by="timestamp",ascending=True)

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
97862,29835,160216,2018-06-18 16:10:00,Reagan Blakey,Chunky Monkey 24 oz,Naturally-Smoothies,Paid,1.0,7.99,0.0,0.66,8.65,8.65,0.0,0.00,2517,General Family Membership
97851,29824,160232,2018-06-18 18:30:00,Kailey Jonas,Chunky Monkey 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,1478,General Single Membership
97762,29735,160281,2018-06-19 10:10:00,Abigail Kelly,Chunky Monkey 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.6,0.44,5.83,5.83,0.0,0.00,2262,General Family Membership
97763,29736,160281,2018-06-19 10:10:00,Abigail Kelly,Chunky Monkey 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.6,0.44,5.83,5.83,0.0,0.00,2262,General Single Membership
97790,29763,160312,2018-06-19 15:11:00,Jessica Barger,Tropical Light 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,1229,General Family Membership
97802,29775,160312,2018-06-19 15:11:00,Jessica Barger,Tropical Light 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,1229,General Property Owner
97801,29774,160313,2018-06-19 15:11:00,Jessica Barger,Chunky Monkey 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,1229,General Family Membership
97800,29773,160313,2018-06-19 15:11:00,Jessica Barger,Chunky Monkey 16 oz,Naturally-Smoothies,Paid,1.0,5.99,0.0,0.49,6.48,6.48,0.0,0.00,1229,General Property Owner
98465,30438,160344,2018-06-20 09:34:00,Lance Wright,Chunky Monkey 24 oz,Naturally-Smoothies,Paid,1.0,7.99,0.0,0.66,8.65,8.65,0.0,0.00,193,General Family Membership
98464,30437,160345,2018-06-20 10:08:00,Anna Cohn,Chunky Monkey 24 oz,Naturally-Smoothies,Paid,1.0,7.99,0.0,0.66,8.65,8.65,0.0,0.00,77,General Family Membership


In [22]:
member[member.item_group == "Membership Initiation Liability"]

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
101999,2390,228075,2019-09-11 15:17:00,Julie Willeke,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,5171,General Family Membership
102175,2566,228620,2019-09-13 10:27:00,Enda Jackson,6 for 360 Fitness (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,5176,General Single Membership
103533,3924,225938,2019-08-29 18:38:00,Mandy Carlisle,6 for 360 Fitness (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,29.7,389.7,389.7,0.0,0.0,4950,General Family Membership
103536,3927,225941,2019-08-29 18:41:00,Mandy Carlisle,6 for 360 Fitness (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,4950,General Family Membership
103543,3934,225954,2019-08-29 18:55:00,Timothy Huynh,6 for 360 Fitness (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,4809,General Family Membership
103544,3935,225955,2019-08-29 18:55:00,Cara Berry,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,3170,General Family Membership
103545,3936,225956,2019-08-29 18:56:00,Chelsea Rogers,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,5100,General Family Membership
114779,15170,232688,2019-10-03 14:40:00,James Black,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,4953,General Family Membership
115108,15499,230369,2019-09-25 12:50:00,Jessica Scott,6 for 360 Tennis (6 pack),Membership Initiation Liability,Paid,1.0,360.0,0.0,0.0,360.0,360.0,0.0,0.0,3355,General Family Membership


---

In [92]:
dft = df.sort_values("timestamp").set_index("timestamp")

In [ ]:
sns.lineplot(x=dft.index,y="total",hue="product_category",data=dft)